In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
! pip install glove-python-binary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 948 kB 8.0 MB/s 


In [3]:
from glove import Glove
glove=Glove()
glove_model=glove.load("/content/drive/MyDrive/FYP/IMDb/Glove/glove.model")

In [4]:
import pandas as pd
import numpy as np
df=pd.read_csv("/content/drive/MyDrive/FYP/IMDb/IMDB-preprocessed.csv")
y=df["sentiment"]


In [5]:
maxi=-1
for i,rev in enumerate(df['new_text']):
  tokens=rev.split()
  if(len(tokens)>maxi):
    maxi=len(tokens)
print(maxi)

2511


In [6]:

import tensorflow as tf
maxlen=66
max_words=len(glove_model.dictionary)
from tensorflow.keras.preprocessing.sequence import pad_sequences
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=max_words)
def padded_sequence(X):
  tokenizer.fit_on_texts(X)
  sequences = tokenizer.texts_to_sequences(X)
  data = pad_sequences(sequences, maxlen=maxlen)
  return data
X=padded_sequence(df["new_text"])

In [7]:
import numpy as np
embed_matrix=np.zeros(shape=(max_words,300))
for word,i in tokenizer.word_index.items():
  if word in glove_model.dictionary:
    embed_vector=glove_model.word_vectors[glove_model.dictionary[word]]
  # word is in the vocabulary learned by the w2v model
    embed_matrix[i]=embed_vector

In [ ]:
embed_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.08644114, -0.46788544,  0.19239497, ...,  0.43217102,
        -0.3802913 ,  0.18661583],
       [ 0.03671082, -0.37766596, -0.04600333, ...,  0.34606747,
        -0.33929718, -0.09252944],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [9]:
from tensorflow.keras import Sequential
from tensorflow.keras.initializers import Constant
from tensorflow.keras.layers import ReLU
from tensorflow.keras.layers import Dropout,Embedding,GRU,Dense
model=Sequential()
model.add(Embedding(input_dim=max_words,output_dim=300,input_length=66,embeddings_initializer=Constant(embed_matrix)))
model.add(GRU(128,return_sequences=True))
model.add(GRU(128,return_sequences=False)) 
model.add(Dense(32,activation='relu'))
model.add(Dropout(0.60))
model.add(Dense(16,activation='relu'))
model.add(Dropout(0.60))
model.add(Dense(1,activation='sigmoid'))

In [10]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=5, batch_size=64, verbose=1)
model_json = model.to_json()
with open("gru.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("gru.h5")
print("Saved model to disk")

Epoch 1/5
586/586 [==============================] - 28s 33ms/step - loss: 0.5350 - accuracy: 0.7155
Epoch 2/5
586/586 [==============================] - 19s 33ms/step - loss: 0.2857 - accuracy: 0.8955
Epoch 3/5
586/586 [==============================] - 19s 32ms/step - loss: 0.1335 - accuracy: 0.9591
Epoch 4/5
586/586 [==============================] - 19s 33ms/step - loss: 0.0557 - accuracy: 0.9832
Epoch 5/5
586/586 [==============================] - 20s 34ms/step - loss: 0.0297 - accuracy: 0.9906
Saved model to disk


In [ ]:
y_pred=model.predict(X_test)
for i in range(len(y_pred)):
  if y_pred[i]>0.5:
    y_pred[i]=1
  else:
    y_pred[i]=0

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
def display_metrics(y_test,y_pred):
    print(accuracy_score(y_test, y_pred))
    print(f1_score(y_test, y_pred, average="macro"))
    print(precision_score(y_test, y_pred, average="macro"))
    print(recall_score(y_test, y_pred, average="macro"))
display_metrics(y_test,y_pred)

0.82512
0.8250734096320975
0.8250904048352762
0.825059838529112
